# Introduction 

This demonstration page is to provide a learning scaffold for the upcoming IDS Deep Dive session, this is to be accompanied by a practical worksheet which users can complete at their own pace after this session. We encourage users to follow along with us within the session, and having a go at the various exercises within the parrellel worksheet following the session. 

## Section 0: Setup of Workbench Space

To start this practical, we first must setup our workbench space within Qwiki-labs environment, this is a step one will not need to complete within the IDS environment, however for today's practical we will. For this we can follow the steps highlighted in Task 1 of the Lab. 

When we've followed all those steps, and we've reached Task 2, we can then import this repo from git, using the command

<code> !git clone https://github.com/wisejthomas/ids_deepdive_demo </code>

This will import this practical and demo into this workspace! 

### Important Definitions 

At this point, it is important to cover a few critical definitions:

- **BigQuery** BigQuery is a fully managed, AI-ready data platform that helps you manage and analyze your data with built-in features like machine learning, search, geospatial analysis, and business intelligence. BigQuery's serverless architecture lets you use languages like SQL and Python to answer your organization's biggest questions with zero infrastructure management.

- **JupyterLab** JupyterLab is a highly extensible, feature-rich notebook authoring application and editing environment, and is a part of Project Jupyter, a large umbrella project centered around the goal of providing tools (and standards) for interactive computing with computational notebooks.

- **Vertex AI Workbench instance** Vertex AI Workbench instances are Jupyter notebook-based development environments on Google Cloud for the entire data science workflow. Vertex AI Workbench instances are prepackaged with JupyterLab. Vertex AI Workbench instances have integrations and features can make it easier to access your data, process data faster, schedule notebook runs, and more.

For most of these methods, we will use a Python JupyterLab Kernel. In the Jupyter architecture, kernels are separate processes started by the server that run your code in different programming languages and environments. We will use the IPython Jupyter Kernel that comes prepackaged with Vertex AI Workbench Instances. This will allow us to execute Python code in this notebook interactively.


## Section 1: Navigating Data in BigQuery Studio 

To explore that data available to you within your project space, we can use the BigQuery Studio space. This is a UI on Google Cloud, which allows your to view information such as column names and descriptions, as well as a preview of the data. 

To access this UI you can select this link 
- Link: https://console.cloud.google.com/bigquery 
- Navigate via the UI to BigQuery >> Studio

Here we can begin to explore the public data, in particular, the London Fire Brigade call outs in 2017. We can explore this data by following these steps: 

- Select project "bigquery-public-data" (if you don't see it, don't panic, just search for "*London*" in the search bar and it should appear!)
    - Locate and select the dataset: "london_fire_brigade"
        - Select table: "fire_brigade_service_calls"
            - We can then preview a sample of this table, as well as view information such as column descriptions.

## Section 2: Introducing Jupyter Lab for Python 



### Section 2a: Package Setup and Installation

In [2]:
# Install a pip package in the current Jupyter kernel
# here the package is bigframes https://pypi.org/project/bigframes/
import sys
!{sys.executable} -m pip install bigframes | grep -v 'already satisfied'

import bigframes.pandas as bpd
import matplotlib.pyplot as plt
import pandas as pd

# import warnings filter & ignore all future warnings
# this is for teaching purposes only, to avoid FutureWarnings to do with bigframe compiler implementation
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Section 3: Supercharging Python with "BigFrames"

### Section 3a: Load in Data
For today's demonstration we will be exploring the Google Dataset around London Fire Brigade call outs in 2017. This dataset has been selected due to its similarity in variable type to the data. 

To demonstrate the potential power when using BigFrames, we will first load in the data, before using magic commands to manipulate the data then creating descriptive statistics and running simple crosstabs to gain a better understanding of the data available. 


In [3]:
# Load data from BigQuery
print("step 1-of-3 :: loading data from BigQuery")
query_or_table = "bigquery-public-data.london_fire_brigade.fire_brigade_service_calls"
bq_df = bpd.read_gbq(query_or_table, use_cache=False)

step 1-of-3 :: loading data from BigQuery


In [4]:
# Lets double check our column names 
list(bq_df)

['incident_number',
 'date_of_call',
 'cal_year',
 'time_of_call',
 'hour_of_call',
 'timestamp_of_call',
 'incident_group',
 'stop_code_description',
 'special_service_type',
 'property_category',
 'property_type',
 'address_qualifier',
 'postcode_full',
 'postcode_district',
 'borough_code',
 'borough_name',
 'proper_case',
 'ward_code',
 'ward_name',
 'ward_name_new',
 'easting_m',
 'northing_m',
 'easting_rounded',
 'northing_rounded',
 'frs',
 'incident_station_ground',
 'first_pump_arriving_attendance_time',
 'first_pump_arriving_deployed_from_station',
 'second_pump_arriving_attendance_time',
 'second_pump_arriving_deployed_from_station',
 'num_stations_with_pumps_attending',
 'num_pumps_attending']

### Section 3b: Using Magic Commands to Manipulate Data
In order to improve the effeciency of writting queries and performing particular tasks, BigQuery has developed "cell magics" to make it easy to execute SQL queries. There are two BigQuery "cell magics": 

> %%bigquery
- Behind the scenes, the %%bigquery magic command uses the BigQuery client library for Python to run the given query
- Then convert the results to a pandas DataFrame
- Then display results.

> %%bigquery my_pandas_data_frame
- Behind the scenes, the %%bigquery magic command uses the BigQuery client library for Python to run the given query
- Then convert the results to a pandas DataFrame
- Then save the pandas DataFrame to the variable my_pandas_data_frame

Let us first of all consider, how many call outs, for the fire service were made by hour during 2017. We can assign this to the data frame *hour_of_call_incident_count* - we will come back to this later when we look to visualise these results! 

In [5]:
%%bigquery hour_of_call_incident_count
SELECT 
    hour_of_call,
    COUNT(DISTINCT incident_number) as incident_number_count
FROM 
    bigquery-public-data.london_fire_brigade.fire_brigade_service_calls
GROUP BY 
    hour_of_call
ORDER BY 
    hour_of_call ASC;


Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
# In the meantime however, we can view the first five hours of the day!
hour_of_call_incident_count.head()

,hour_of_call,incident_number_count
0,0,976
1,1,879
2,2,645
3,3,549
4,4,485


### Section 3c: Basic Descriptive Statistics

Exploring the basic descriptive statistics of any given dataset is critical to beginning to understand its components and what you, as an analyst, can achieve with it. Luckily, Python includes a basic descriptive function, <code> .describe() </code> , which can be applied simply to BigFrame. 

For this we will explore the numerical variables within our dataset, in particular the hour of call, number of pumps attending and the number of stations with pumps attending.

In [7]:
%%bigquery fire_numerical_describe
SELECT 
    hour_of_call,
    num_stations_with_pumps_attending,
    num_pumps_attending
FROM 
    bigquery-public-data.london_fire_brigade.fire_brigade_service_calls

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
# Run the describe function to explore the core basic descriptive statistics
fire_numerical_describe.describe()

,hour_of_call,num_stations_with_pumps_attending,num_pumps_attending
count,32247.0,32179.0,32179.0
mean,13.465222,1.356537,1.536375
std,6.152739,0.6114,0.741248
min,0.0,1.0,1.0
25%,9.0,1.0,1.0
50%,14.0,1.0,1.0
75%,18.0,2.0,2.0
max,23.0,6.0,7.0


### Section 3d: Building Cross Tabs 

Cross tabs can be used to examine the relationship between two categorical variables. This can be extremely beneficial to understand the frequency of different occuring factors and to begin to further understand the data presented. 

When building cross tabs, the easiest method for medium sized datasets at present is utilizing Pandas, on your local memory. This is because at present the same function is yet to be fully available within the BigFrames library. 

For this example, we will look at the relationship between Incident Group and Property Category. 

In [9]:
%%bigquery crosstab_explore_data
SELECT 
    incident_group,
    property_category
FROM 
    bigquery-public-data.london_fire_brigade.fire_brigade_service_calls

Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
# Crosstab in Pandas 
## https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html
pd.crosstab(
    crosstab_explore_data.incident_group, 
    crosstab_explore_data.property_category)

property_category,Aircraft,Boat,Dwelling,Non Residential,Other Residential,Outdoor,Outdoor Structure,Rail Vehicle,Road Vehicle
incident_group,,,,,,,,,
False Alarm,5,3,7032,6089,1455,558,253,4,333
Fire,0,9,1856,721,113,1383,1604,4,744
Special Service,20,12,6352,959,228,653,147,10,1700


## Section 4: Visualising your Data in Python
Lets return to our number of incident per hour, from Section 3b. Lets plot this using matplotlib.

When using matplotlib we can simply highlight our X and Y variables respectively. In this case, we would like to create a bar plot, between the "hour of calls" and the "number of incidents" in that time. 

In [ ]:
# Bar Chart Call X = Hour of Call, Y = Incident Number Count 
plt.bar(hour_of_call_incident_count.hour_of_call, hour_of_call_incident_count.incident_number_count)
# Label our X and Y axis and Title 
plt.xlabel('Hour of Call')
plt.ylabel('Number of Incidents')
plt.title('Number of Incidents recorded per Hour')